In [1]:
!pip install mediapipe opencv-python


In [4]:
!pip install pyttsx3

# SHOULDER PRESS


In [3]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3

# Initialize Mediapipe Pose.
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Start capturing video from webcam.
cap = cv2.VideoCapture(0)  # Change index if you have multiple cameras.

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Set the OpenCV window to be resizable.
cv2.namedWindow('Double Arm Shoulder Press Tracking', cv2.WINDOW_NORMAL)

# Maximize the window.
cv2.setWindowProperty('Double Arm Shoulder Press Tracking', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Initialize the text-to-speech engine.
engine = pyttsx3.init()

# Initialize variables to track exercise state.
stage = "initial"
feedback_given = False  # To prevent repetitive feedback
reps = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Convert the image to RGB.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and find pose.
        results = pose.process(image)

        # Convert the image back to BGR.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of shoulder, elbow, and wrist for both arms.
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate angles at the elbows for both arms.
            left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            # Visualize the angles.
            cv2.putText(image, str(int(left_angle)),
                        tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA
                        )
            cv2.putText(image, str(int(right_angle)),
                        tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA
                        )

            # Check for initial L shape position
            if stage == "initial" and 80 < left_angle < 100 and 80 < right_angle < 100:
                stage = "start"
                engine.say("Start position detected")
                engine.runAndWait()

            # Provide corrective feedback during exercise
            if stage == "start":
                if left_angle > 160 and right_angle > 160:
                    stage = "up"
                    feedback_given = False

                if left_angle < 160 or right_angle < 160:
                    if not feedback_given:
                        engine.say("Push your arms up fully.")
                        engine.runAndWait()
                        feedback_given = True

            if stage == "up" and left_angle < 90 and right_angle < 90:
                stage = "down"
                reps += 1
                engine.say(f"Shoulder press done correctly. Repetition count: {reps}")
                engine.runAndWait()
                stage = "initial"  # Reset stage for the next repetition

        except:
            pass

        # Render pose landmarks on the image.
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the output.
        cv2.imshow('Double Arm Shoulder Press Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# BICEP CURL

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3

# Initialize Mediapipe Pose.
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Start capturing video from webcam.
cap = cv2.VideoCapture(0)  # Change index if you have multiple cameras.

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Set the OpenCV window to be resizable.
cv2.namedWindow('Bicep Curl Tracking', cv2.WINDOW_NORMAL)

# Maximize the window.
cv2.setWindowProperty('Bicep Curl Tracking', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Initialize the text-to-speech engine.
engine = pyttsx3.init()

# Initialize variables to track exercise state.
stage = None
feedback_given = False  # To prevent repetitive feedback

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Convert the image to RGB.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and find pose.
        results = pose.process(image)

        # Convert the image back to BGR.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of shoulder, elbow, and wrist.
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angle at the elbow.
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize the angle.
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA
                        )

            # Provide corrective feedback.
            if angle > 160 and stage == "up":
                if not feedback_given:
                    engine.say("Curl your arm fully.")
                    engine.runAndWait()
                    feedback_given = True
            elif angle < 160:
                stage = "up"  # Transition to 'up' stage
                feedback_given = False
            if angle < 30 and stage == "up":
                stage = "down"  # Transition to 'down' stage
                engine.say("Bicep curl done correctly")
                engine.runAndWait()

        except:
            pass

        # Render pose landmarks on the image.
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the output.
        cv2.imshow('Bicep Curl Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# ALTERNATE BICEP CURL

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3

# Initialize Mediapipe Pose.
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize the text-to-speech engine.
engine = pyttsx3.init()

def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Start capturing video from webcam.
cap = cv2.VideoCapture(0)  # Change index if you have multiple cameras.

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Set the OpenCV window to be resizable.
cv2.namedWindow('Alternating Bicep Curl Tracking', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Alternating Bicep Curl Tracking', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Initialize variables to track exercise state.
left_stage = "initial"
right_stage = "initial"
current_arm = "left"  # Start with the left arm
reps = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Convert the image to RGB.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and find pose.
        results = pose.process(image)

        # Convert the image back to BGR.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of shoulder, elbow, and wrist for both arms.
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate angles at the elbows for both arms.
            left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            # Visualize the angles.
            cv2.putText(image, f"Left angle: {int(left_angle)}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Right angle: {int(right_angle)}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Enforce alternating sequence: start with the left arm
            if current_arm == "left":
                if left_stage == "initial" and left_angle > 160:
                    left_stage = "start"
                    engine.say("Start position detected for left arm. Curl your arm fully.")
                    engine.runAndWait()

                if left_stage == "start" and left_angle < 60:
                    left_stage = "up"

                if left_stage == "up" and left_angle > 160:
                    left_stage = "initial"
                    reps += 1
                    engine.say(f"Left arm bicep curl done correctly. Repetition count: {reps}")
                    engine.runAndWait()
                    current_arm = "right"

            elif current_arm == "right":
                if right_stage == "initial" and right_angle > 160:
                    right_stage = "start"
                    engine.say("Start position detected for right arm. Curl your arm fully.")
                    engine.runAndWait()

                if right_stage == "start" and right_angle < 60:
                    right_stage = "up"

                if right_stage == "up" and right_angle > 160:
                    right_stage = "initial"
                    reps += 1
                    engine.say(f"Right arm bicep curl done correctly. Repetition count: {reps}")
                    engine.runAndWait()
                    current_arm = "left"

        except Exception as e:
            print(e)

        # Render pose landmarks on the image.
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the output.
        cv2.imshow('Alternating Bicep Curl Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# TRICEP KICKBACK EXERCISE

In [12]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3

# Initialize Mediapipe Pose.
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize the text-to-speech engine.
engine = pyttsx3.init()

def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Start capturing video from webcam.
cap = cv2.VideoCapture(0)  # Change index if you have multiple cameras.

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Set the OpenCV window to be resizable.
cv2.namedWindow('Tricep Kickback Tracking', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Tricep Kickback Tracking', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Initialize variables to track exercise state.
left_stage = "initial"
right_stage = "initial"
current_arm = "left"  # Start with the left arm
reps = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Convert the image to RGB.
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and find pose.
        results = pose.process(image)

        # Convert the image back to BGR.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of shoulder, elbow, and wrist for both arms.
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate angles at the elbows for both arms.
            left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            # Visualize the angles.
            cv2.putText(image, f"Left angle: {int(left_angle)}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f"Right angle: {int(right_angle)}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Enforce alternating sequence: start with the left arm
            if current_arm == "left":
                if left_stage == "initial" and left_angle < 30:
                    left_stage = "start"
                    engine.say("Start position detected for left arm. Extend your arm fully.")
                    engine.runAndWait()

                if left_stage == "start" and left_angle > 150:
                    left_stage = "up"

                if left_stage == "up" and left_angle < 30:
                    left_stage = "initial"
                    reps += 1
                    engine.say(f"Left arm tricep kickback done correctly. Repetition count: {reps}")
                    engine.runAndWait()
                    current_arm = "right"

            elif current_arm == "right":
                if right_stage == "initial" and right_angle < 30:
                    right_stage = "start"
                    engine.say("Start position detected for right arm. Extend your arm fully.")
                    engine.runAndWait()

                if right_stage == "start" and right_angle > 150:
                    right_stage = "up"

                if right_stage == "up" and right_angle < 30:
                    right_stage = "initial"
                    reps += 1
                    engine.say(f"Right arm tricep kickback done correctly. Repetition count: {reps}")
                    engine.runAndWait()
                    current_arm = "left"

        except Exception as e:
            print(e)

        # Render pose landmarks on the image.
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the output.
        cv2.imshow('Tricep Kickback Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
